In [54]:
#resnet_transf.py
#Faz transfer learning usando ResNet50.
#https://medium.com/abraia/first-steps-with-transfer-learning-for-custom-image-classification-with-keras-b941601fcad5
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import numpy as np;
from tensorflow import keras 
import keras.backend as K
from keras import optimizers, callbacks, regularizers
from keras.regularizers import l2
from keras.models import Sequential, Model
from keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input, decode_predictions
from keras.utils import load_img, img_to_array

In [55]:
#Descarregar segm_eliret.zip
url='http://www.lps.usp.br/hae/apostila/feiCorCrop.zip'
import os; nomeArq=os.path.split(url)[1]
if not os.path.exists(nomeArq):
  print("Baixando o arquivo",nomeArq,"para diretorio default",os.getcwd())
  os.system("wget -nc -U 'Firefox/50.0' "+url)
else:
  print("O arquivo",nomeArq,"ja existe no diretorio default",os.getcwd())
print("Descompactando arquivos novos de",nomeArq)  
os.system("unzip -u "+nomeArq) 

Baixando o arquivo feiCorCrop.zip para diretorio default d:\Faculdade\PSI3472\Aula_10
Descompactando arquivos novos de feiCorCrop.zip


1

In [56]:
def leCsv(nomeDir, nomeArq, nl=0, nc=0):
    st=os.path.join(nomeDir,nomeArq);
    arq=open(st,"rt"); lines=arq.readlines(); arq.close(); n=len(lines)
    
    linhas_separadas=[]
    for linha in lines:
        linha=linha.strip('\n'); linha=linha.split(';'); linhas_separadas.append(linha)
    
    ay=np.empty((n),dtype='float32'); ax=np.empty((n,nl,nc,3),dtype='float32')
    for i in range(len(linhas_separadas)):
        linha=linhas_separadas[i]
        img_path=os.path.join(nomeDir,linha[0])
        t = load_img(img_path, target_size=(nl,nc))
        x = img_to_array(t)
        x = np.expand_dims(x, axis=0)
        ax[i] = preprocess_input(x)
        ay[i] = np.float32(linha[1]) #0=m ou 1=f
    return ax, ay

In [57]:
nomeprog="resnet_transf"
#Original: 280x200, redimensionado: 224x224
num_classes=2; nl=224; nc=224
batch_size = 10
diretorioBd="Dados"
ax, ay = leCsv(diretorioBd,"treino.csv", nl=nl, nc=nc); #200 imagens
qx, qy = leCsv(diretorioBd,"teste.csv", nl=nl, nc=nc); #100 imagens
vx, vy = leCsv(diretorioBd,"valida.csv", nl=nl, nc=nc); #100 imagens
ay = keras.utils.to_categorical(ay, num_classes)
qy = keras.utils.to_categorical(qy, num_classes)
vy = keras.utils.to_categorical(vy, num_classes)

In [58]:
def modelevaluate(model,ax,ay,vx,vy,qx,qy):
    score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
    score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
    score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)

In [59]:
from keras.models import load_model
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'models/resnet_transf' + str(i) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

In [60]:
members = load_all_models(5)
print('Loaded %d models' % len(members))

>loaded models/resnet_transf0.h5
>loaded models/resnet_transf1.h5
>loaded models/resnet_transf2.h5
>loaded models/resnet_transf3.h5
>loaded models/resnet_transf4.h5
Loaded 5 models


In [61]:
for model in members:
	modelevaluate(model,ax,ay,vx,vy,qx,qy)

Training loss: [0.09653446823358536, 0.9900000095367432]
Validation loss: [0.9457689523696899, 0.9599999785423279]
Test loss: [0.15690910816192627, 0.9800000190734863]
Training loss: [0.028125612065196037, 0.9950000047683716]
Validation loss: [0.7118646502494812, 0.949999988079071]
Test loss: [0.08846602588891983, 0.9900000095367432]
Training loss: [6.098863741499372e-05, 1.0]
Validation loss: [1.0286810398101807, 0.9300000071525574]
Test loss: [0.011690777726471424, 0.9900000095367432]
Training loss: [0.05391884222626686, 0.9900000095367432]
Validation loss: [2.0524823665618896, 0.9100000262260437]
Test loss: [0.003270217450335622, 1.0]
Training loss: [0.23480074107646942, 0.9850000143051147]
Validation loss: [0.4384792447090149, 0.9599999785423279]
Test loss: [0.062085919082164764, 0.9700000286102295]


In [68]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, testX):
	# make predictions
	yhats = [model.predict(testX) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	result = np.argmax(summed, axis=1)
	return result

from sklearn.metrics import accuracy_score
def evaluate_n_members(members, n_members, testX, testy):
	# select a subset of members
	subset = members[:n_members]
	print(len(subset))
	# make prediction
	yhat = ensemble_predictions(subset, testX)
	# calculate accuracy
	return accuracy_score(testy, yhat)

In [69]:
n_members = 6
scores = list()
for i in range(1, n_members+1):
	score = evaluate_n_members(members, i, qx, qy)
	print('> %.3f' % score)
	scores.append(score)
# plot score vs number of ensemble members
x_axis = [i for i in range(1, n_members+1)]

import matplotlib.pyplot as plt
plt.plot(x_axis, scores)
plt.show()

1
4/4 [==============================] - 7s 1s/step


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets